In [4]:
import pandas as pd
import numpy as np
from IPython.display import display
import os

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


# Paso 1: LECTURA DE DATAFRAME

In [6]:
df=pd.read_csv(os.path.join('..','DataFrames','users_behavior.csv'))

In [8]:
display(df.head(5))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [9]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

In [10]:
display(df.describe())

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


# Paso 2: La segmentación de los datos en conjuntos de entrenamiento, validación y prueba.

In [11]:
#DATOS SEGMENTADOS
X=df.drop('is_ultra',axis=1) #DATOS A TOMAR EN CUENTA EN ENTRAMIENTO
y=df['is_ultra'] #OBJETIVO
#PRUEBA 30% ENTRENAMIENTO 70%
X_train, X_temp, y_train, y_temp = train_test_split(X,y, test_size=0.3, random_state=42)
#PRUEBA 30% ---- 50% VALIDACION Y EL OTRO 50% PRUEBA
X_val, X_test, y_val, y_test = train_test_split(X_temp,y_temp, test_size=0.5, random_state=42)


In [15]:
print(f'Datos Entrenamiento: {X_train.shape}')
print(f'Datos Prueba: {X_test.shape}')
print(f'Datos Validacion: {X_val.shape}')


Datos Entrenamiento: (2249, 4)
Datos Prueba: (483, 4)
Datos Validacion: (482, 4)


# Paso 3: Exploración de Modelos y Evaluación

In [25]:
max_depth_values=[5,10,15,20,25]

n_estimators=100
random_state=42

param_grid={
    'max_depth': max_depth_values,
    'n_estimators':[n_estimators],
    'random_state':[random_state]
}

base_model = RandomForestClassifier()

gridsearch=GridSearchCV(estimator=base_model, param_grid=param_grid, scoring='accuracy', cv=5)
gridsearch.fit(X_train, y_train)

resultados=gridsearch.cv_results_


In [26]:
print(f'RESULTADOS BUSQUEDA HIPERPARAMETROS:')
for ms, p in zip(resultados['mean_test_score'], resultados['params']):
    print(f'Exactitud:{ms:.4f}\nParametros: {p}')

RESULTADOS BUSQUEDA HIPERPARAMETROS:
Exactitud:0.7990
Parametros: {'max_depth': 5, 'n_estimators': 100, 'random_state': 42}
Exactitud:0.8066
Parametros: {'max_depth': 10, 'n_estimators': 100, 'random_state': 42}
Exactitud:0.8043
Parametros: {'max_depth': 15, 'n_estimators': 100, 'random_state': 42}
Exactitud:0.7995
Parametros: {'max_depth': 20, 'n_estimators': 100, 'random_state': 42}
Exactitud:0.8003
Parametros: {'max_depth': 25, 'n_estimators': 100, 'random_state': 42}


Elegiremos la combinacion de max_depth 10, n_estimators 100, y random state 42 ya que nos da una exactitud de 80.66%

In [28]:
best_model= gridsearch.best_estimator_
print(F'Mejores Parametros: {gridsearch.best_params_}')



Mejores Parametros: {'max_depth': 10, 'n_estimators': 100, 'random_state': 42}
RandomForestClassifier(max_depth=10, random_state=42)


In [31]:
y_val_p=best_model.predict(X_val)
exactitud= accuracy_score(y_val, y_val_p)
print(f'Exactitud: {exactitud:.4f}')


Exactitud: 0.8237


# Paso 4:Comprueba la calidad del modelo usando el conjunto de prueba.

In [39]:
y_test_p=best_model.predict(X_test)
Test_exactitud= accuracy_score(y_test, y_test_p)
print(f'Test_Exactitud: {Test_exactitud:.4f}\nMejor Modelo:{best_model.__class__.__name__}')


Test_Exactitud: 0.7805
Mejor Modelo:RandomForestClassifier


# Paso 5: Tarea adicional: haz una prueba de cordura al modelo. Estos datos son más complejos que los que habías usado antes así que no será una tarea fácil. 

In [40]:
y_test_pred_cordura = best_model.predict(X_test) + np.random.normal(0, 0.1, len(X_test))

y_test_pred_cordura_binarizada = np.where(y_test_pred_cordura >= 0.5, 1, 0)

Cordura_exactitud= accuracy_score(y_test, y_test_pred_cordura_binarizada)

print(f'Cordura_Exactitud: {Cordura_exactitud:.4f}\nMejor Modelo:{best_model.__class__.__name__}')

Cordura_Exactitud: 0.7805
Mejor Modelo:RandomForestClassifier
